In [1]:
import duckdb
import pandas as pd 

# Create a new DuckDB database on disk
con = duckdb.connect('sample_trades_compare.duckdb')




In [2]:
# Create tables from CSV files
con.execute("CREATE TABLE trades1 AS SELECT * FROM read_csv_auto('sample_trades.csv')")
con.execute("CREATE TABLE trades2 AS SELECT * FROM read_csv_auto('sample_trades_2.csv')")

In [3]:


con.execute("ALTER TABLE trades1 ADD PRIMARY KEY (trade_id);")
con.execute("ALTER TABLE trades2 ADD PRIMARY KEY (trade_id);")

In [4]:
con.execute("select count(*) from trades1").fetch_df()

,count_star()
0,43000000


In [5]:
query = """
SELECT 
    t1.trade_id,
    t1.timestamp as timestamp1,
    t2.timestamp as timestamp2,
    (t1.timestamp = t2.timestamp)::INT as timestamp_match,
    t1.symbol as symbol1,
    t2.symbol as symbol2,
    (t1.symbol = t2.symbol)::INT as symbol_match,
    t1.quantity as quantity1,
    t2.quantity as quantity2,
    (t1.quantity = t2.quantity)::INT as quantity_match,
    t1.price as price1,
    t2.price as price2,
    (t1.price = t2.price)::INT as price_match
FROM trades1 t1
INNER JOIN trades2 t2 ON t1.trade_id = t2.trade_id
WHERE t1.timestamp != t2.timestamp 
   OR t1.symbol != t2.symbol
   OR t1.quantity != t2.quantity 
   OR t1.price != t2.price
ORDER BY t1.trade_id
"""

differences_df = con.execute(query).fetch_df()
differences_df



,trade_id,timestamp1,timestamp2,timestamp_match,symbol1,symbol2,symbol_match,quantity1,quantity2,quantity_match,price1,price2,price_match
0,20,2024-05-14 20:34:03,2024-05-14 20:34:03,1,JPM,JPM,1,3161,3161,1,278.40,630.55,0
1,27,2024-05-20 20:34:03,2024-05-20 20:34:03,1,TSLA,TSLA,1,7108,7108,1,75.48,291.45,0
2,103,2024-09-12 20:34:03,2024-09-12 20:34:03,1,META,META,1,8477,8477,1,743.42,267.60,0
3,109,2024-06-24 20:34:03,2024-06-24 20:34:03,1,WMT,WMT,1,3848,3848,1,230.93,687.88,0
4,168,2024-05-20 20:34:03,2024-05-20 20:34:03,1,BAC,BAC,1,7522,7522,1,31.50,426.13,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
885262,42999712,2024-12-08 20:39:35,2024-12-08 20:39:35,1,AMZN,AMZN,1,5385,5385,1,140.56,802.90,0
885263,42999741,2024-12-07 20:39:35,2024-12-07 20:39:35,1,GOOGL,GOOGL,1,7819,7819,1,771.78,892.03,0
885264,42999875,2024-08-21 20:39:35,2024-08-21 20:39:35,1,MSFT,MSFT,1,2981,2981,1,955.72,937.41,0
885265,42999931,2024-06-04 20:39:35,2024-06-04 20:39:35,1,GOOGL,GOOGL,1,5083,5083,1,406.03,775.75,0


In [6]:

# Create summary DataFrame showing counts of matches/mismatches
match_summary = pd.DataFrame({
    'Field': ['Timestamp', 'Symbol', 'Quantity', 'Price'],
    'Matches': [
        differences_df['timestamp_match'].sum(),
        differences_df['symbol_match'].sum(),
        differences_df['quantity_match'].sum(),
        differences_df['price_match'].sum()
    ],
    'Mismatches': [
        len(differences_df) - differences_df['timestamp_match'].sum(),
        len(differences_df) - differences_df['symbol_match'].sum(),
        len(differences_df) - differences_df['quantity_match'].sum(),
        len(differences_df) - differences_df['price_match'].sum()
    ]
})

match_summary

,Field,Matches,Mismatches
0,Timestamp,885267,0
1,Symbol,885267,0
2,Quantity,885267,0
3,Price,0,885267
